### Table 4: PTdetector Detection Score Ranking of Libraries

The related experiment data is arranged as follow.

`detection_result.csv` @ The detection result after applying PTdetector on 80 top-traffic web pages.
- `domain`: The domain name
- `rank`: The domain rank
- `url`: The home page url under the domain
- `ground truth`: The ground truth library loaded on the web page
- `PTdetector`: The libraries identified by PTdetector

Following code is used to decode library string format.

In [26]:
def convertToObject(str_list):
    # <libname> @ <version> : <score> ! <depth>
    obj_list = []
    for str in str_list:
        obj = {}
        _depth_pos = str.find('!')
        if _depth_pos != -1:
            obj['depth'] = int(str[_depth_pos + 1:])
            str = str[:_depth_pos]
        else:
            obj['depth'] = None

        _colon_pos = str.find(':')
        if _colon_pos != -1:
            obj['score'] = float(str[_colon_pos + 1:])
            str = str[:_colon_pos]
        else:
            obj['score'] = None

        _at_pos = str.find('@')
        if _at_pos != -1:
            obj['version'] = str[_at_pos + 1:]
            str = str[:_at_pos]
        else:
            obj['version'] = None

        obj['name'] = str

        obj_list.append(obj)
    return obj_list

Then calculate the score ranking of all libraries.

In [41]:
import json
import pandas as pd
from functools import cmp_to_key

df= pd.read_csv("./detection_result.csv",delimiter=';')
def collectScores(column_name):
    # Collect the scores of each library occuring in ground truth column
    score_list = []
    gt_column = df['ground truth'].tolist()
    tool_column = df[column_name].tolist()

    for i in range(len(tool_column)):
        list0 = convertToObject(json.loads(gt_column[i]))
        list1 = convertToObject(json.loads(tool_column[i]))


        for lib_info in list0:
            lib_name = lib_info['name']
            lib_score = 0
            for lib_info2 in list1:
                if lib_info2['name'] == lib_info['name']:
                    if lib_info2['score'] != None:
                        lib_score = lib_info2['score']
                    else:
                        lib_score = 100
                    break
            find_lib_in_list = False
            for lib_info3 in score_list:
                if lib_info3['name'] == lib_info['name']:
                    find_lib_in_list = True
                    lib_info3['scores'].append(lib_score)
                    break
            if not find_lib_in_list:
                score_list.append({"name":lib_name, "scores":[lib_score]})

    return score_list

def compareScore(a, b):
    # Compare first based on 'score', then on 'cnt'
    if a['avg'] < b['avg']:
        return 1
    elif  a['avg'] == b['avg']:
        if a['cnt'] < b['cnt']:
            return 1
    return -1

def presentScore(score_list):
    # Sort
    for lib_info in score_list:
        lib_info['avg'] = sum(lib_info['scores']) / len(lib_info['scores'])
        lib_info['cnt'] = len(lib_info['scores'])
    score_list.sort(key=cmp_to_key(compareScore))

    # Print result
    print("No.\tLibrary\t\tavg. score\tcnt")
    for i in range(len(score_list)):
        lib_info = score_list[i]
        libname_text = lib_info['name']
        if len(libname_text) < 8:
            libname_text += '\t'
        print(f"{i+1}\t{libname_text}\t{round(lib_info['avg'],1)}\t\t{lib_info['cnt']}")
        # print(lib_info['scores'])

if __name__ == "__main__":
    score_list = collectScores('PTdetector')
    presentScore(score_list)

No.	Library		avg. score	cnt
1	Lo-Dash		100.0		21
2	IfVisible.js	100.0		4
3	WebFont Loader	100.0		1
4	Head JS		100.0		1
5	Kendo UI	99.9		1
6	Prototype	99.8		1
7	Pusher		99.4		2
8	jQuery		99.1		60
9	RequireJS	99.0		12
10	Moment.js	98.7		6
11	Highcharts	98.5		1
12	Handlebars	98.2		5
13	Zepto.js	96.8		1
14	AngularJS	96.1		2
15	Isotope		95.5		1
16	Underscore	94.4		37
17	core-js		93.9		51
18	Hammer.js	93.5		8
19	Moment Timezone	91.9		3
20	yepnope		91.7		2
21	AMP		91.0		1
22	FlexSlider	90.3		2
23	SPF		89.5		1
24	jQuery UI	89.5		16
25	jQuery Tools	88.9		1
26	Velocity.js	88.5		1
27	SWFObject	88.1		1
28	Backbone	87.8		8
29	Knockout	82.1		1
30	Bootstrap	79.5		7
31	Ext JS		75.4		1
32	Preact		74.7		1
33	Modernizr	65.0		13
34	React		11.4		28
35	Vue		0.0		3
36	Angular		0.0		1
